In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo
import pytz
import time
from IPython.display import clear_output
from pandas import Timestamp
import sell_price_action_detector
import buy_price_action_detector
import order_manager

with open("parameters.txt", "r") as file:
    exec(file.read())

In [2]:
# Initialize MetaTrader 5 connection
order_manager.log_in_to_mt5()

if not mt5.terminal_info().trade_allowed:
    print("❌ Algo Trading is OFF! Please enable it manually.")
else:
    print("✅ Algo Trading is ON")

Connected to MT5 successfully
✅ Algo Trading is ON


In [3]:
#Telegram notification
import nest_asyncio
import asyncio
from telegram import InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CallbackQueryHandler
from datetime import datetime, timedelta, timezone, date

with open("parameters.txt", "r") as file:
    exec(file.read())
decision = None

async def alarm_and_decide():
    global decision, BIAS
    # Apply the patch for nested asyncio event loops
    nest_asyncio.apply()
    
    user_input = None  # Variable to store the decision
    
    app = Application.builder().token(chatbot_token).build()  # Replace with your actual bot token
    
    # Create the buttons for "Yes" and "No"
    keyboard = [
        [InlineKeyboardButton("Yes", callback_data='yes')],
        [InlineKeyboardButton("No", callback_data='no')]
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    
    # Send the message with buttons
    await app.bot.send_message(chat_id=chat_id, text=f"🚨 {symbol} {BIAS} entry signal detected! 🚨 \n {datetime.now().strftime("%Y-%m-%d %I:%M %p")}\n   Take the trade?", reply_markup=reply_markup)
    
    async def handle_callback_query(update, context):
        global decision
        decision = update.callback_query.data  # Store the decision ("yes" or "no")
        
        # Acknowledge the callback
        await update.callback_query.answer()
    
        # Stop the polling loop immediately
        asyncio.get_event_loop().stop()  # Stop the event loop after the callback
        message_id = update.callback_query.message.message_id
        await context.bot.edit_message_reply_markup(chat_id=chat_id, message_id=message_id, reply_markup=None)
        await context.bot.send_message(chat_id=chat_id, text=f"Your decision: {decision}")
    app.add_handler(CallbackQueryHandler(handle_callback_query))
    
    try:
        await app.run_polling()
    except (RuntimeError, RuntimeWarning) as RTE_RTW:
        pass

async def alarm_for_deletion():
    # Apply the patch for nested asyncio event loops
    nest_asyncio.apply()

    app = Application.builder().token(chatbot_token).build()  # Replace with your actual bot token

    # Send the message
    await app.bot.send_message(
        chat_id=chat_id, 
        text=f"🚨 {symbol} {BIAS} trade order will be deleted! 🚨 \n {datetime.now().strftime('%Y-%m-%d %I:%M %p')}\n  P4 was invalidated!"
    )

In [ ]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame_input):
    rates_frame_input['time'] = pd.to_datetime(rates_frame_input['time'], unit='s')
    rates_frame_input.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame_input["time"] = pd.to_datetime(rates_frame_input["time"])
    return rates_frame_input

def append_new_data(rates_frame, rates_frame_to_append):
    rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
    return rates_frame

def download_recent_OHLCdata():
    global symbol
    timeframe = mt5.TIMEFRAME_M1
    
    #GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
    utc_from = datetime.now() - timedelta(minutes=10) + timedelta(hours=2)
    utc_to = datetime.now() + timedelta(hours=2) - timedelta(minutes=1)
    startdate=utc_from.strftime('%Y%m%d')
    enddate=utc_to.strftime('%Y%m%d')
    
    # Get the OHLC data FOR TODAY AND YESTERDAY
    rates_10m = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    # Check if the data is downloaded successfully
    if rates is None:
        print("No data available, error code =", mt5.last_error())
    else:
        # Convert the data to a pandas DataFrame
        rates_frame_to_append = pd.DataFrame(rates_10m)
        df_convert_hr(rates_frame_to_append)
        return rates_frame_to_append

def download_updated_values():
    global rates_frame
    rates_frame_to_append = download_recent_OHLCdata()
    rates_frame = append_new_data(rates_frame, rates_frame_to_append)

def display_updated_values():
    global rates_frame, N, symbol
    print(f"Number of trades executed for today: {N}")
    print("  ")
    print(f"{symbol} OHLC prices from the last 20 minutes :")
    print("____________________________________________________________________________")
    print(rates_frame.tail(20))
    return

list_of_p3x = []
def check_p3_if_it_is_first_time_encountered(p3):
    p3x = p3['time'].iloc[0]
    global list_of_p3x
    if p3x in list_of_p3x:
        p3_is_first_time_encountered = False
    elif p3x not in list_of_p3x:
        p3_is_first_time_encountered  = True
        list_of_p3x.append(p3x)
    return p3_is_first_time_encountered

list_of_p3x_multiple = []
def check_p3_if_it_is_first_time_encountered_multiple(p3):
    global list_of_p3x_multiple
    n = len(p2)
    p3x = p3['time'].iloc[0]
    count = list_of_p3x_multiple.count(p3x)

    if count >= n-1:
        p3_is_first_time_encountered = False
    elif count < n-1:
        p3_is_first_time_encountered  = True
        list_of_p3x_multiple.append(p3x)
    return p3_is_first_time_encountered

#Detect the entry drills
def check_buy_entry_drill():
    if buy_tp == "-":
        Buy = False
        SL_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame

    #check if pattern occured during pre-london session
    pre_ldn_session = pre_ldn_session_start_time <= LowestPoint["time"].to_pydatetime().hour < pre_ldn_session_end_time
    #check if pattern occured during london session:
    ldn_session = ldn_session_start_time <= LowestPoint["time"].to_pydatetime().hour < ldn_session_end_time
    #check if pattern occured during pre-ny session
    pre_ny_session = pre_ny_session_start_time <= LowestPoint["time"].to_pydatetime().hour < pre_ny_session_end_time
    #check if pattern occured during ny session
    ny_session = ny_session_start_time <= LowestPoint["time"].to_pydatetime().hour < ny_session_end_time
    
    condition1 = ldn_session and (round(ldn_low,pip_precision) == round(LowestPoint["low"],pip_precision))
    condition2 = ny_session and (round(ny_low,pip_precision) == round(LowestPoint["low"],pip_precision))
    condition3 = pre_ldn_session and (round(pre_ldn_low,pip_precision) == round(LowestPoint["low"],pip_precision))
    condition4 = pre_ny_session and (round(pre_ny_low,pip_precision) == round(LowestPoint["low"],pip_precision))
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Buy, SL_price, 
            TP1_price, OB_size,
            Entry_price, SL_size,
            p1, p2, p2_bos, p3, p4
        ) = buy_price_action_detector.validate_and_buy(rates_frame_p3, p3_rates_frame, buy_tp)
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Buy = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

def check_sell_entry_drill():
    if sell_tp == "-":
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame
    
    #check if pattern occured during pre-london session
    pre_ldn_session = pre_ldn_session_start_time <= HighestPoint["time"].to_pydatetime().hour < pre_ldn_session_end_time
    #check if pattern occured during london session:
    ldn_session = ldn_session_start_time <= HighestPoint["time"].to_pydatetime().hour < ldn_session_end_time
    #check if pattern occured during pre-ny session
    pre_ny_session = pre_ny_session_start_time <= HighestPoint["time"].to_pydatetime().hour < pre_ny_session_end_time
    #check if pattern occured during ny session
    ny_session = ny_session_start_time <= HighestPoint["time"].to_pydatetime().hour < ny_session_end_time

    condition1 = ldn_session and (round(ldn_high,pip_precision) == round(HighestPoint["high"],pip_precision))
    condition2 = ny_session and (round(ny_high,pip_precision) == round(HighestPoint["high"],pip_precision))
    condition3 = pre_ldn_session and (round(pre_ldn_high,pip_precision) == round(HighestPoint["high"],pip_precision))
    condition4 = pre_ny_session and (round(pre_ny_high,pip_precision) == round(HighestPoint["high"],pip_precision))
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Sell, SL_price, TP1_price, 
            OB_size, Entry_price, SL_size, 
            p1, p2, p2_bos, p3, p4 
        ) = sell_price_action_detector.validate_and_sell(rates_frame_p3, p3_rates_frame, sell_tp)
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

In [ ]:
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
utc_from = datetime.now() - timedelta(days=1)
utc_to = datetime.now() - timedelta(minutes=1) + timedelta(hours=2)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data FOR TODAY AND YESTERDAY
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    rates_frame = df_convert_hr(rates_frame)
    print(rates_frame)

In [ ]:
try:
    current_price = round(float(rates_frame["close"].tail(1).iloc[0]),pip_precision)
    print(f"The current price of {symbol} is about: {current_price}.")

    while True:
        try: 
            # Check for a dash
            if sell_tp == "-":
                print("You are only looking to buy.")
                BEARISH_BIAS = False
                break
        
            sell_tp = float(sell_tp)
            if sell_tp >= current_price:
                print(f"Enter a price that is less than the current price of {symbol}.")
                break
            value = float(sell_tp)
            print(f"You entered a SELL TP level: {value}")
            BEARISH_BIAS = True
            break
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            break
    
    while True:
        try:
            if buy_tp == "-":
                print("You are only looking to sell.")
                BULLISH_BIAS = False
                break
            
            buy_tp = float(buy_tp) 
            if buy_tp <= current_price:
                print(f"Enter a price that is more than the current price of {symbol}.")
                break
                
            value = float(buy_tp)
            print(f"You entered a BUY TP level: {value}")
            BULLISH_BIAS = True
            break
            
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            break
            
except IndexError as err1:
    print(f'IndexError : {err1} ')
except NameError as err2:
    print(f'Name error 1 : {err2} ')

In [ ]:
#number of trades for today
N = order_manager.count_number_of_trades(symbol)
date_today = datetime.now(timezone(timedelta(hours=2))).date()

while N < 4:
    if N >= 3:
        clear_output(wait=True)
        print("You've made 3 trades for the day! Come back tomorrow!")
        break
    else:
        clear_output(wait=True)
        N = order_manager.count_number_of_trades(symbol)
        
        try:
            download_updated_values()
        except (IndexError, KeyError, NameError, ValueError) as error:
            print(f'Error: {error} ')
            print("Sleeping for 60s and will be trying again......")
            time.sleep(60)
            continue
        
        print(f'{symbol}...The date and time today is: {datetime.now().strftime("%Y-%m-%d %I:%M %p")}')
        #HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
        pre_ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= pre_ldn_session_start_time) & (rates_frame['time'].dt.hour < pre_ldn_session_end_time)]
        if len(pre_ldn_rates_frame) != 0:    
            pre_ldn_high = round(pre_ldn_rates_frame["high"].max(),pip_precision)
            pre_ldn_low = round(pre_ldn_rates_frame["low"].min(),pip_precision)
            print("__________________________________")
            print(f'Pre-London high: {pre_ldn_high}')
            print(f'Pre-London low:  {pre_ldn_low}')
        else: 
            pass

        ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= ldn_session_start_time) & (rates_frame['time'].dt.hour < ldn_session_end_time)]
        if len(ldn_rates_frame) != 0:
            ldn_high = round(ldn_rates_frame["high"].max(),pip_precision)
            ldn_low = round(ldn_rates_frame["low"].min(),pip_precision)
            print(f'\nLondon high:.....{ldn_high}')
            print(f'London low:......{ldn_low}')
        else:
            pass

        #HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
        pre_ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= pre_ny_session_start_time) & (rates_frame['time'].dt.hour < pre_ny_session_end_time)]
        if len(pre_ny_rates_frame) != 0:
            pre_ny_high = round(pre_ny_rates_frame["high"].max(),pip_precision)
            pre_ny_low = round(pre_ny_rates_frame["low"].min(),pip_precision)
            print(f'\nPre-NY high:.....{pre_ny_high}')
            print(f'Pre-NY low:......{pre_ny_low}')
        else:
            pass
            
        ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= ny_session_start_time) & (rates_frame['time'].dt.hour < ny_session_end_time)]
        if len(ny_rates_frame) != 0:    
            ny_high = round(ny_rates_frame["high"].max(),pip_precision)
            ny_low = round(ny_rates_frame["low"].min(),pip_precision)
            print(f'\nNY high:.........{ny_high}')
            print(f'NY low:..........{ny_low}')
        else:
            pass
        print("__________________________________")
        order_manager.show_orders_and_open_positions()
        
        try:
            #Determine the max and min value of the highs and lows for the last 35 mins
            Last_110 = rates_frame.tail(110)
            MaxHighPrice = Last_110["high"].max()
            MinLowPrice = Last_110["low"].min()
    
            #Determine the if the last 80 mins is in a downtrend or uptrend
            HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
            LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]
        except IndexError as err1:
            print("It is weekend you dumbass! The forex market is closed!")
            
        #Based on the trend (and the conditions inside) decide whether to buy, sell, or wait.
        try:
            if (HighestPoint["time"] > LowestPoint["time"]) & BEARISH_BIAS:
                BIAS = "sell"
                print("Looking to sell...\n")
                MaxHighPoint = Last_110[Last_110["high"] == MaxHighPrice].iloc[[0]]
                MaxHighTime = MaxHighPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MaxHighTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MaxHighTime]
                Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_sell_entry_drill()
                p1 = p1.drop_duplicates(subset=["time", "price"])
                p2 = p2.drop_duplicates(subset=["time", "price"])
                num_rows_p2 = p2.shape[0]
                p3 = p3.drop_duplicates(subset=["time", "price"])
                p3 = p3.loc[p3.index.repeat(num_rows_p2)].reset_index(drop=True)
                
                print(f'Sell: {Sell}')
                
                if Sell == False:
                    if (len(p1) == 0) and (len(p2) == 0) and (len(p3) == 0):
                        print("Waiting for a proper higher high and higher low to form....\n")
                    elif (len(p1) != 0) and (len(p2) != 0) and (len(p3) != 0):
                        print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3} \n-----------------------------------')
                        print("Waiting for a proper BOS...\n")
                    else:
                        pass
                elif Sell == True:
                    p4_is_valid= sell_price_action_detector.verify_p4(p2_bos,p3,p4,p3_rates_frame)
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        sell_orders = []
                    else:
                        # Filter only sell orders
                        sell_orders = [order for order in orders if order.type in [3, 5]]
                        
                    current_time = datetime.now() + timedelta(hours=2)
                    two_hours_ago = current_time - timedelta(hours=2)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    two_hours_ago_timestamp = int(two_hours_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        sell_positions = []
                    else:
                        # Filter only buy orders
                        sell_positions = [
                            position for position in positions
                            if position.type == 1  # Buy position (0 = BUY, 1 = SELL)
                            and two_hours_ago_timestamp <= position.time <= current_timestamp]
                        
                    print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3}\n\np4: {p4}\n-----------------------------------')
                    if len(sell_positions) == 0:
                        if p4_is_valid and (len(sell_orders) == 0):
                            p3_is_first_time_encountered = check_p3_if_it_is_first_time_encountered(p3)
                            if p3_is_first_time_encountered == True:
                                await alarm_and_decide()
                                if decision == 'yes':
                                    print("P4 is valid...")
                                    tick = mt5.symbol_info_tick(symbol)
                                    ask_price = tick.ask
                                    if ask_price > Entry_price:
                                        order_manager.place_sell_stop_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                                    elif ask_price < Entry_price:
                                        order_manager.place_sell_limit_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                                elif decision == 'no':
                                    print("Wait for other sell setups")
                            elif p3_is_first_time_encountered == False:
                                print("Wait for other sell setups")
                        elif p4_is_valid and (len(sell_orders) != 0):
                            sell_order_entry = sell_orders[0].price_open if sell_orders else None
                            if Entry_price == sell_order_entry:
                                print("Waiting for either entry or another BOS....")
                            elif Entry_price != sell_order_entry:
                                p3_is_first_time_encountered = check_p3_if_it_is_first_time_encountered_multiple(p3.head(1))
                                if p3_is_first_time_encountered == True:
                                    await alarm_and_decide()
                                    await alarm_for_deletion()
                                    print("Deleting the existing buy order....")
                                    order_manager.delete_order(symbol)
                                    if decision == 'yes':
                                        print("P4 is valid...")
                                        tick = mt5.symbol_info_tick(symbol)
                                        ask_price = tick.ask
                                        if ask_price > Entry_price:
                                            order_manager.place_sell_stop_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                                        elif ask_price < Entry_price:
                                            order_manager.place_sell_limit_order(symbol,Entry_price, SL_price, TP1_price, sell_tp)
                                    elif decision == 'no':
                                        print("Wait for other sell setups")
                                elif p3_is_first_time_encountered == False:
                                    print("Wait for other sell setups")
                        elif (p4_is_valid == False) and (len(sell_orders) != 0):
                            print("The point p4 got invalidated...")
                            print(f"Found {len(sell_orders)} pending orders.")
                            print("Deleting the existing sell order....")
                            await alarm_for_deletion()
                            order_manager.delete_order(symbol)
                            
                        elif (p4_is_valid == False):
                            print("P4 is invalid. Waiting for another set up.")
                    elif sell_positions != 0: #if there is an open position, have a trailing stop to the top of P5 to be safer.
                        print(f"{symbol} trade in progress...")
                        positions = mt5.positions_get(symbol=symbol)
                        if positions is None:
                            positions = []
                        else:
                            # Filter only sell positions
                            sell_positions = [positions for positions in positions if positions.type in [1]]

                        for position in sell_positions:
                            entry_price = position.price_open
                            stop_loss = position.sl
                            tick = mt5.symbol_info_tick(symbol)
                            ask_price = tick.ask
                            new_stop_loss = round(entry_price - breathing_room,pip_precision)
                            SL_size = abs(round(stop_loss - entry_price,pip_precision))
                            current_RR = (entry_price - ask_price) / SL_size

                            if abs(entry_price - stop_loss) <= breathing_room*2:
                                if current_RR >=5:
                                    trail_stop_loss(symbol, new_stop_loss)
                                    print(f"Stop loss of {symbol} sell position moved to breakeven!")
                else:
                    print("Dayummm why do you have two p4s??? ")
                    break

                orders = mt5.orders_get(symbol=symbol)
                sell_orders = [order for order in orders if order.type in [3, 5]]
                if (len(sell_orders) != 0):
                    print("Waiting for entry...")    
                
                display_updated_values()
                time.sleep(60)
                N = order_manager.count_number_of_trades(symbol)
                continue
            elif (HighestPoint["time"] < LowestPoint["time"]) & BULLISH_BIAS:
                BIAS = "buy"
                print("Looking to buy...\n")
                MinLowPoint = Last_110[Last_110["low"] == MinLowPrice].iloc[[0]]
                MinLowTime = MinLowPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MinLowTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MinLowTime]                
                Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_buy_entry_drill()
                p1 = p1.drop_duplicates(subset=["time", "price"])
                p2 = p2.drop_duplicates(subset=["time", "price"])
                num_rows_p2 = p2.shape[0]
                p3 = p3.drop_duplicates(subset=["time", "price"])
                p3 = p3.loc[p3.index.repeat(num_rows_p2)].reset_index(drop=True)
                
                print(f'Buy: {Buy}')

                if Buy == False:
                    if (len(p1) == 0) and (len(p2) == 0) and (len(p3) == 0):
                        print("Waiting for a proper lower low and lower high to form....\n")
                    elif (len(p1) != 0) and (len(p2) != 0) and (len(p3) != 0):
                        print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3} \n-----------------------------------')
                        print("Waiting for a proper BOS...\n")
                    else:
                        pass
                elif Buy == True:
                    p4_is_valid = buy_price_action_detector.verify_p4(p2_bos,p3,p4,p3_rates_frame)
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        buy_orders = []
                    else:
                        # Filter only buy orders
                        buy_orders = [order for order in orders if order.type in [2, 4]]

                    current_time = datetime.now() + timedelta(hours=2)
                    two_hours_ago = current_time - timedelta(hours=2)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    two_hours_ago_timestamp = int(two_hours_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        buy_positions = []
                    else:
                        # Filter only buy orders
                        buy_positions = [
                            position for position in positions
                            if position.type == 0  # Buy position (0 = BUY, 1 = SELL)
                            and two_hours_ago_timestamp <= position.time <= current_timestamp]

                    print(f'-----------------------------------\np1: {p1}\n\np2: {p2}\n\np3: {p3}\n\np4: {p4}\n-----------------------------------')

                    if len(buy_positions) == 0:
                        if p4_is_valid and (len(buy_orders) == 0):
                            p3_is_first_time_encountered = check_p3_if_it_is_first_time_encountered(p3)
                            if p3_is_first_time_encountered == True:
                                await alarm_and_decide()
                                if decision == 'yes':
                                    print("P4 is valid...")
                                    tick = mt5.symbol_info_tick(symbol)
                                    bid_price = tick.bid
                                    if bid_price > Entry_price:
                                        order_manager.place_buy_limit_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                                    elif bid_price < Entry_price:
                                        order_manager.place_buy_stop_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                                elif decision == 'no':
                                    print("Wait for other buy setups!")
                            elif p3_is_first_time_encountered == False:
                                print("Wait for other buy setups!")
                        elif p4_is_valid and (len(buy_orders) != 0):
                            buy_order_entry = buy_orders[0].price_open if buy_orders else None
                            if Entry_price == buy_order_entry:
                                print("Waiting for either entry or another BOS....")
                            elif Entry_price != buy_order_entry:
                                p3_is_first_time_encountered = check_p3_if_it_is_first_time_encountered_multiple(p3.head(1))
                                if p3_is_first_time_encountered == True:
                                    await alarm_and_decide()
                                    await alarm_for_deletion()
                                    print("Deleting the existing buy order....")
                                    order_manager.delete_order(symbol)
                                    if decision == 'yes':
                                        print("P4 is valid...")
                                        tick = mt5.symbol_info_tick(symbol)
                                        bid_price = tick.bid
                                        if bid_price > Entry_price:
                                            order_manager.place_sell_stop_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                                        elif bid_price < Entry_price:
                                            order_manager.place_sell_limit_order(symbol,Entry_price, SL_price, TP1_price, buy_tp)
                                    elif decision == 'no':
                                        print("Wait for other buy setups")
                                elif p3_is_first_time_encountered == False:
                                    print("Wait for other buy setups")              
                        elif (p4_is_valid == False) and (len(buy_orders) != 0):
                            print("The point p4 got invalidated...")
                            print(f"Found {len(buy_orders)} pending orders.")
                            print("Deleting the existing buy order....")
                            await alarm_for_deletion()
                            order_manager.delete_order(symbol)
                            
                        elif (p4_is_valid == False):
                            print("P4 is invalid. Waiting for another set up.")
                    elif len(buy_positions) != 0: #if there is an open position, have a trailing stop to the top of P5 to be safer.
                        print(f"{symbol} trade in progress..")
                        positions = mt5.positions_get(symbol=symbol)
                        if positions is None:
                            positions = []
                        else:
                            # Filter only sell positions
                            buy_positions = [positions for positions in positions if positions.type in [0]]

                        for position in buy_positions:
                            entry_price = position.price_open
                            stop_loss = position.sl
                            tick = mt5.symbol_info_tick(symbol)
                            bid_price = tick.bid
                            new_stop_loss = round(entry_price + breathing_room,pip_precision)
                            SL_size = abs(round(entry_price - stop_loss,pip_precision))
                            current_RR = (bid_price - entry_price) / SL_size

                            if abs(entry_price - stop_loss) <= breathing_room*2:
                                if current_RR >=5:
                                    trail_stop_loss(symbol, new_stop_loss)
                                    print(f"Stop loss of {symbol} buy position moved to breakeven!")
                else:
                    print("Dayummm why do you have two p4s??? ")
                    break

                orders = mt5.orders_get(symbol=symbol)
                buy_orders = [order for order in orders if order.type  in [2, 4]]
                if (len(buy_orders) != 0):
                    print("Waiting for entry...")
                    
                display_updated_values()
                time.sleep(60)
                N = order_manager.count_number_of_trades(symbol)
                continue
            else:
                if BULLISH_BIAS:
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        buy_orders = []
                    else:
                        # Filter only buy orders
                        buy_orders = [order for order in orders if order.type in [2, 4]]
                    if (len(buy_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(buy_orders)} pending orders.")
                        print('This is a no trade scenario....')
                        order_manager.delete_order(symbol)


                    current_time = datetime.now() + timedelta(hours=2)
                    two_hours_ago = current_time - timedelta(hours=2)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    two_hours_ago_timestamp = int(two_hours_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        buy_positions = []
                    else:
                        # Filter only buy orders
                        buy_positions = [
                            position for position in positions
                            if position.type == 0  # Buy position (0 = BUY, 1 = SELL)
                            and two_hours_ago_timestamp <= position.time <= current_timestamp]
                    if len(buy_positions) != 0:
                        print("Buy trade in progress...")
                    else:
                        print("No trade scenario...")
                elif BEARISH_BIAS:
                    orders = mt5.orders_get(symbol=symbol)
                    if orders is None:
                        sell_orders = []
                    else:
                        # Filter only sell orders
                        sell_orders = [order for order in orders if order.type in [3, 5]]
                    if (len(sell_orders) != 0):
                        print("The point p4 got invalidated...")
                        print(f"Found {len(sell_orders)} pending orders.")
                        print('This is a no trade scenario....')
                        order_manager.delete_order(symbol)


                    current_time = datetime.now() + timedelta(hours=2)
                    two_hours_ago = current_time - timedelta(hours=2)
                    
                    # Convert times to timestamps (seconds since epoch)
                    current_timestamp = int(current_time.timestamp())
                    two_hours_ago_timestamp = int(two_hours_ago.timestamp())
                    
                    positions = mt5.positions_get(symbol=symbol)
                    if positions is None:
                        sell_positions = []
                    else:
                        # Filter only buy orders
                        sell_positions = [
                            position for position in positions
                            if position.type == 1  # Buy position (0 = BUY, 1 = SELL)
                            and two_hours_ago_timestamp <= position.time <= current_timestamp]
                    if len(sell_positions) != 0:
                        print("Sell trade in progress...")
                    else:
                        print("No trade scenario...")
                display_updated_values()
                time.sleep(60)
                continue

        except (IndexError, KeyError, NameError, ValueError) as error:
            print(f'Error: {error} ')
            print("Sleeping for 60s and will be trying again......")
            time.sleep(60)
            continue